# FIFA 23 PREDICTIVE AI

In [1]:
# AUTHOR: Kobina Kyereboah-Coleman
# DATE: June 23rd 2024
# TOD: Assignment 2
# TODO: Player Rating Prediction AI

## PYTHON LIBRARIES & MODULES

In [2]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

## DATA PREP

In [3]:
# Load the dataset
fifa_23 = pd.read_csv('Male Players Legacy.csv')

<ipython-input-3-cd4fa369f29b>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_23 = pd.read_csv('Male Players Legacy.csv')


In [4]:
# View the first 5 rows of the dataframe
fifa_23.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [5]:
fifa_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


## DATA CLEANING

In [6]:
### NA Threshold Function
# A function that takes a dataframe and returns the features that are safe to work with
def threshold(data):
    # Renaming dataset for conveniece
    # data = fifa_23

    # The data integrity threshold provided (30%)
    threshold = 0.3

    # Calculating threshold count for missing values
    threshold_count = int(threshold * len(data))

    # Pulling out unsafe columns
    unsafe_columns = [col for col in data.columns if data[col].isnull().sum() > threshold_count]
    print(f"{len(unsafe_columns)} unsafe columns were identified.")

    return unsafe_columns

In [7]:
# Calling the threshold function on my dataframe
threshold(fifa_23)

8 unsafe columns were identified.


['club_loaned_from',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'release_clause_eur',
 'player_tags',
 'player_traits',
 'goalkeeping_speed']

In [8]:
# Dropping the unsafe features(columns) in the dataframe
dataset = fifa_23.drop(columns = ['club_loaned_from','nation_team_id','nation_position','nation_jersey_number','release_clause_eur','player_tags','player_traits','goalkeeping_speed'])

In [9]:
dataset.columns
# The number of coolumns in the dataframe is now '102'.

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=102)

In [10]:
# Dropping extra unecessary columns that are irrelevant to player rating
dataset = dataset.drop(columns = ['fifa_update','player_id','player_url','player_positions','dob','fifa_update_date','work_rate','body_type','real_face','short_name','long_name','club_contract_valid_until_year','fifa_version','league_level','nationality_id','player_face_url','league_id', 'league_name','club_team_id', 'club_name', 'club_position', 'club_jersey_number','club_joined_date', 'nationality_name', 'preferred_foot', 'weak_foot','ls', 'st', 'rs','lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm','rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb','rcb', 'rb', 'gk'])
dataset.columns

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'age', 'height_cm',
       'weight_kg', 'skill_moves', 'international_reputation', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
       'goalkeeping_diving', 'goalkeeping_handling', '

In [11]:
# Identifying Columns With Missing Values
# A function that searches for columns in the dataframe with missing values
def find_null_columns(x):
    null_columns = []
    for col in x.columns:
        if x[col].isnull().any():
            null_columns.append(col)
    print(null_columns)

In [12]:
find_null_columns(dataset)
# The displayed list is an array of the columns with missing entriess

['value_eur', 'wage_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'mentality_composure']


In [13]:
# Handling missing values in the null columns
# Initializing an array to store the columns with null values
columns_to_rectify = ['value_eur', 'wage_eur', 'league_id', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_joined_date', 'club_contract_valid_until_year', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'mentality_composure']

# A function that replaces missing values in the columns of a datset with the deian of the column
for column in columns_to_rectify:
    if column in dataset.columns:
        # Fill NAs in each column with the median of that particular column
        dataset[column] = dataset[column].fillna(dataset[column].median())

## CORRELATION MATRIX

In [14]:
correlation_matrix = dataset.select_dtypes(include=[np.number]).corr()

In [15]:
# Mapping correlation on the overall column
# This enables the inspection of how the overall rating of players varies with other columns
corr_overall = correlation_matrix['overall'].sort_values(ascending=False)

In [16]:
corr_overall

overall                        1.000000
movement_reactions             0.845753
potential                      0.695362
passing                        0.629868
wage_eur                       0.607297
mentality_composure            0.600033
value_eur                      0.562677
dribbling                      0.561034
attacking_short_passing        0.499757
mentality_vision               0.493282
international_reputation       0.485234
skill_long_passing             0.483808
power_shot_power               0.480297
physic                         0.473469
age                            0.458390
skill_ball_control             0.457011
shooting                       0.444746
skill_curve                    0.415863
power_long_shots               0.409086
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
attacking_volleys              0.377048
skill_dribbling                0.369827
power_stamina                  0.360025


In [17]:
dataset.shape

(161583, 49)

## FEATURE ENGINEERING

#### Feature engineering is the process of creating new features or transforming existing features in a dataset to improve the performance of machine learning models. This involves using domain knowledge, creativity, and various techniques to extract useful information from raw data that can enhance the predictive power of the model.

In [18]:
# Aggregating statistics for mental
mentality_stats = ['mentality_composure','mentality_aggression','mentality_positioning','mentality_vision','mentality_penalties','mentality_interceptions']
dataset['mental'] = dataset[mentality_stats].mean(axis = 1)
dataset.drop(columns = mentality_stats, inplace = True)

In [19]:
# Aggregating statistics for power
power_stats = ['power_shot_power','power_long_shots','power_stamina','power_long_shots','power_strength','power_jumping']
dataset['power'] = dataset[power_stats].mean(axis = 1)
dataset.drop(columns = power_stats, inplace = True)

In [20]:
# Aggregating statistics for skill
skill_stats = ['skill_fk_accuracy','skill_long_passing','skill_ball_control','skill_curve','skill_dribbling','skill_moves']
dataset['skill'] = dataset[skill_stats].mean(axis=1)
dataset.drop(columns = skill_stats, inplace = True)

In [21]:
# Aggregating statistics for goal keeping
goal_keeping_stats = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking','goalkeeping_positioning', 'goalkeeping_reflexes']
dataset['goalkeeping'] = dataset[goal_keeping_stats].mean(axis=1)
dataset.drop(columns = goal_keeping_stats, inplace=True)

In [22]:
# Aggregating statistics for movement
movement_stats = ['movement_acceleration','movement_sprint_speed','movement_agility','movement_reactions','movement_balance']
dataset['movement'] = dataset[movement_stats].mean(axis = 1)
dataset.drop(columns = movement_stats, inplace = True)

In [23]:
# Aggregating statistics for attack
attacking_stats = ['attacking_crossing','attacking_finishing','attacking_heading_accuracy','attacking_short_passing','attacking_volleys']
dataset['attack'] = dataset[attacking_stats].mean(axis = 1)
dataset.drop(columns = attacking_stats, inplace = True)

In [24]:
# Aggregating statistics for defence
defending_stats = ['defending_marking_awareness','defending_standing_tackle','defending_sliding_tackle']
dataset['defence'] = dataset[defending_stats].mean(axis = 1)
dataset.drop(columns = defending_stats, inplace = True)

In [25]:
dataset.info()
dataset.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   overall                   161583 non-null  int64  
 1   potential                 161583 non-null  int64  
 2   value_eur                 161583 non-null  float64
 3   wage_eur                  161583 non-null  float64
 4   age                       161583 non-null  int64  
 5   height_cm                 161583 non-null  int64  
 6   weight_kg                 161583 non-null  int64  
 7   international_reputation  161583 non-null  int64  
 8   pace                      161583 non-null  float64
 9   shooting                  161583 non-null  float64
 10  passing                   161583 non-null  float64
 11  dribbling                 161583 non-null  float64
 12  defending                 161583 non-null  float64
 13  physic                    161583 non-null  f

(161583, 21)

## TRAINING

In [26]:
# Target variable (y) = 'overall' (player's overall rating)
y = dataset['overall']
X = dataset.drop('overall', axis=1)  # all variables excluding 'overall'

In [27]:
sc = StandardScaler()
scaled = sc.fit_transform(X)
X = pd.DataFrame(scaled, columns=X.columns)

In [28]:
y.head()

0    93
1    92
2    90
3    90
4    90
Name: overall, dtype: int64

In [29]:
X.head()

,potential,value_eur,wage_eur,age,height_cm,weight_kg,international_reputation,pace,shooting,passing,dribbling,defending,physic,mental,power,skill,goalkeeping,movement,attack,defence
0,3.875315,16.447428,24.697104,0.401872,-1.813330,-1.176360,10.148137,2.376280,2.783978,2.926570,3.480487,-1.565492,-0.211839,1.038591,1.763595,2.439629,-0.328442,2.725123,2.381346,-1.186189
1,3.396013,12.846177,16.681825,0.830120,0.556996,0.680668,10.148137,2.376280,3.088467,2.421008,2.965866,-1.248264,1.529711,1.260377,2.864128,2.160490,-0.281304,2.037119,2.518145,-1.023979
2,3.076478,8.742427,12.101666,1.044244,-0.183731,0.680668,10.148137,2.376280,2.555612,2.623233,3.068791,-1.248264,-0.102992,1.191069,1.821518,2.185866,-0.387364,2.544069,1.888867,-0.942873
3,3.076478,8.407427,12.101666,1.472491,2.038450,2.823392,10.148137,0.751875,2.936223,2.421008,2.451245,-1.121372,2.291639,1.537608,2.429707,2.020920,-0.269519,0.805956,2.326626,-0.748221
4,3.076478,10.249927,13.246706,0.615996,1.742159,2.394847,10.148137,0.083002,0.119701,0.095424,0.083989,0.274433,0.114701,-1.553525,-0.683640,-1.404881,4.220363,-0.588155,-1.462725,-1.040200


In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   potential                 161583 non-null  float64
 1   value_eur                 161583 non-null  float64
 2   wage_eur                  161583 non-null  float64
 3   age                       161583 non-null  float64
 4   height_cm                 161583 non-null  float64
 5   weight_kg                 161583 non-null  float64
 6   international_reputation  161583 non-null  float64
 7   pace                      161583 non-null  float64
 8   shooting                  161583 non-null  float64
 9   passing                   161583 non-null  float64
 10  dribbling                 161583 non-null  float64
 11  defending                 161583 non-null  float64
 12  physic                    161583 non-null  float64
 13  mental                    161583 non-null  f

In [31]:
y.value_counts()

overall
64    9826
65    9795
66    9645
67    9583
63    8919
68    8848
69    7921
62    7818
70    7466
61    6585
60    6503
71    6315
72    6076
73    5204
59    5014
58    4589
74    4243
75    4147
57    3774
56    3372
76    3000
55    2911
54    2431
77    2297
53    2116
78    1845
52    1804
79    1475
51    1327
80    1162
50    1055
81     820
82     705
49     601
83     517
48     381
84     372
85     231
47     199
86     163
87     129
88      98
46      76
89      69
90      41
45      36
91      24
44      19
92      11
93       7
94       6
42       4
43       3
40       3
41       2
Name: count, dtype: int64

In [32]:
# Splitting my into a training set and test set
# X - training
# y - testing

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

In [33]:
Xtrain.shape

(129266, 20)

## IMPORTING MODELS

In [34]:
  from sklearn.linear_model import LinearRegression
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import GradientBoostingRegressor
  from sklearn.svm import SVR
  from sklearn.neighbors import KNeighborsRegressor
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.ensemble import AdaBoostRegressor
  from sklearn.ensemble import VotingRegressor
  from xgboost import XGBRegressor
  from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [35]:
# A function to train my machine learning model
def train_and_evaluate(model_name, model, Xtrain, Ytrain, Xtest, Ytest):
    model.fit(Xtrain, Ytrain)
    y_pred = model.predict(Xtest)
    mae = mean_absolute_error(Ytest, y_pred)
    mse = mean_squared_error(Ytest, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(Ytest, y_pred)
    print(f"{model_name}:")
    print(f"  Mean Absolute Error: {mae}")
    print(f"  Mean Squared Error: {mse}")
    print(f"  Root Mean Squared Error: {rmse}")
    print(f"  R2 Score: {r2}")
    print()

In [36]:
# Evaluation: Linear Regression
linear_regression = LinearRegression()
train_and_evaluate("Linear Regression", linear_regression, Xtrain, Ytrain, Xtest, Ytest)

Linear Regression:
  Mean Absolute Error: 1.5893643360359617
  Mean Squared Error: 4.170189107509101
  Root Mean Squared Error: 2.042104088314085
  R2 Score: 0.915861671099248



In [37]:
# Evaluation: Decision Tree Regressor
decision_tree = DecisionTreeRegressor()
train_and_evaluate("Decision Tree Regressor", decision_tree, Xtrain, Ytrain, Xtest, Ytest)

Decision Tree Regressor:
  Mean Absolute Error: 0.561066930717579
  Mean Squared Error: 1.203267630039917
  Root Mean Squared Error: 1.0969355633034772
  R2 Score: 0.9757227010569793



In [38]:
# Evaluation: AdaBoost Regressor
ada_boost = AdaBoostRegressor()
train_and_evaluate("AdaBoost Regressor", ada_boost, Xtrain, Ytrain, Xtest, Ytest)

AdaBoost Regressor:
  Mean Absolute Error: 1.6921918319825409
  Mean Squared Error: 4.2027547196317325
  Root Mean Squared Error: 2.0500621257980773
  R2 Score: 0.9152046226745868



In [39]:
# Evaluation: XGB Regressor
xgb_regressor = XGBRegressor()
train_and_evaluate("XGB Regressor", xgb_regressor, Xtrain, Ytrain, Xtest, Ytest)

XGB Regressor:
  Mean Absolute Error: 0.5813361927288263
  Mean Squared Error: 0.6356068344108963
  Root Mean Squared Error: 0.7972495433745298
  R2 Score: 0.9871759060544922



In [40]:
# Evaluation: Random Forest Regressor
random_forest = RandomForestRegressor()
train_and_evaluate("Random Forest Regressor", random_forest, Xtrain, Ytrain, Xtest, Ytest)


KeyboardInterrupt



In [ ]:
# Evaluation: Gradient Boosting Regressor
gradient_boosting = GradientBoostingRegressor()
train_and_evaluate("Gradient Boosting Regressor", gradient_boosting, Xtrain, Ytrain, Xtest, Ytest)

## ENSEMBLE LEARNING

In [ ]:
# Voting Regressor
# Evaluation
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor(),
    "Support Vector Regressor": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "XGB Regressor": XGBRegressor()
}
voting_regressor = VotingRegressor(estimators=list(models.items()))
train_and_evaluate("Voting Regressor", voting_regressor, Xtrain, Ytrain, Xtest, Ytest)

## GRID-SEARCH CROSS VALIDATION

In [ ]:
# Importing pickle file
import pickle as pkl
from sklearn.model_selection import GridSearchCV

In [ ]:
# hyperparameters for Random Forest Regressor
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Hyperparameters for XGBoost Regressor
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Hyperparameters for GBR Regressor
gbr_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.05, 0.01],
    'subsample': [0.8, 0.9, 1.0]
}

## MODEL INITIALIZATION

In [ ]:
models = {
    'Random Forest': (RandomForestRegressor(), rf_params),
    'XGB Regressor': (XGBRegressor(), xgb_params),
    'Gradient Boosting': (GradientBoostingRegressor(), gbr_params)
}

In [ ]:
best_models = {}
for model_name, (model, params) in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
    grid_search.fit(Xtrain, Ytrain)
    best_params = grid_search.best_params_
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {best_params}")

In [ ]:
for model_name, best_model in best_models.items():
    y_pred = best_model.predict(Xtest)
    print(f"Predictions using best {model_name}:", y_pred)

In [ ]:
# Saving the model to a pickle file
with open('/content/regressor_model.pkl', 'wb') as file:
    pkl.dump(regressor, file)